# Langchain4j example

Start by adding the dependencies.

In [ ]:
//DEPS dev.langchain4j:langchain4j-open-ai:1.0.0
//DEPS dev.langchain4j:langchain4j:1.0.0
//DEPS org.slf4j:slf4j-simple:2.0.17

//DEPS io.github.cdimascio:dotenv-java:3.2.0


## Setup env to get API secrets

In [ ]:
import static java.lang.IO.*;
import java.nio.file.*;
import io.github.cdimascio.dotenv.Dotenv;

if(!Files.exists(Paths.get("env.properties"))) {
    println("env.properties not found, please create it and add OPENAI_API_KEY");
}

var env = Dotenv.configure().ignoreIfMissing().filename("env.properties").load();
var apiKey = env.get("OPENAI_API_KEY");

Setup language model:

In [ ]:
import dev.langchain4j.model.openai.OpenAiChatModel;
import static java.time.Duration.ofSeconds;
import dev.langchain4j.agent.tool.Tool;
import dev.langchain4j.service.AiServices;
import dev.langchain4j.memory.chat.MessageWindowChatMemory;

var chatModel = OpenAiChatModel.builder()
                .apiKey(apiKey)
                .modelName("gpt-4o-mini")
                .temperature(0.0)
                .timeout(ofSeconds(60))
                .build();

interface Assistant {
    String chat(String message);
}

var ai = AiServices.builder(Assistant.class)
                .chatModel(chatModel)
                .chatMemory(MessageWindowChatMemory.withMaxMessages(20))
                .build();

Lets talk with the AI:

In [ ]:
display(ai.chat("generate a nice html paragraph with some nice colors and words, emojis and something moving/animating. only return the html, no other text"),"text/html");


## LLM's has limits

For example if we ask it to report how many letters are in a word, it will most likely not get it right.

In [ ]:
ai.chat("how many r's are in in the word 'strawberry'?");

## Use a tool

Lets make a class that has methods that can be used as tools, i.e. to use for counting characters in a word.

In [ ]:
public class Tools {
   
    @Tool
    public long countOccurrences(String word, String character) {
        println("countOccurrences: " + word + " " + character); // put here so we can see the tool is called
        return word.chars().filter(c -> c == character.charAt(0)).count();
    }
}

new Tools().countOccurrences("strawberry", "r");

In [ ]:
var ai = AiServices.builder(Assistant.class)
                .chatModel(chatModel)
                .chatMemory(MessageWindowChatMemory.withMaxMessages(20))
                .tools(new Tools())
                .build();

Now lets ask again...

In [ ]:
ai.chat("how many r's are in in the word 'strawberry'?");